code adpated from homework 2 of DS-GA 1012, source code:
    https://colab.research.google.com/drive/1V_lMTW0m__CeSmMdntUoyriyyrQcAoUJ?authuser=2

## Imports

In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 163 kB 2.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check version number
import imblearn
print(imblearn.__version__)

0.6.2


In [38]:
# imports 
#!pip install sacremoses

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import pandas as pd
import sacremoses
from torch.utils.data import dataloader, Dataset
import tqdm

## Read in data

In [8]:
# read in data
# change path if necessary
train = pd.read_csv (r'/Users/christineshen/Desktop/NYU/Spring 2020/ML 1003/project/train.csv')
dev = pd.read_csv (r'/Users/christineshen/Desktop/NYU/Spring 2020/ML 1003/project/dev.csv')

In [78]:
train.head()

,ex_id,user_id,prod_id,rating,label,date,review
0,0,923,0,3.0,1,2014-12-08,The food at snack is a selection of popular Gr...
1,1,924,0,3.0,1,2013-05-16,This little place in Soho is wonderful. I had ...
2,2,925,0,4.0,1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...
3,3,926,0,4.0,1,2011-07-28,This is a beautiful quaint little restaurant o...
4,4,927,0,4.0,1,2010-11-01,Snack is great place for a casual sit down lu...


In [19]:
# check number of the 2 classes in train and dev 
print('check class imbalance in train set')
print(train.label.value_counts())
print('check class imbalance in dev set')
print(dev.label.value_counts())

check class imbalance in train set
0    225055
1     25819
Name: label, dtype: int64
check class imbalance in dev set
0    32270
1     3648
Name: label, dtype: int64


In [34]:
# split into X_train, X_dev, y_train, y_dev
X_train, y_train = list(train.review),list(train.label)
X_dev, y_dev = list(dev.review),list(dev.label)

## GloVe Embeddings

In [36]:
# === Download Preprocessed version
!wget https://docs.google.com/uc?id=1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu -O glove_split.aa
!wget https://docs.google.com/uc?id=1LF2yD2jToXriyD-lsYA5hj03f7J3ZKaY -O glove_split.ab
!wget https://docs.google.com/uc?id=1N1xnxkRyM5Gar7sv4d41alyTL92Iip3f -O glove_split.ac
!cat glove_split.?? > 'glove.6B.300d__50k.txt'

--2020-05-19 14:25:18--  https://docs.google.com/uc?id=1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu
Resolving docs.google.com (docs.google.com)... 2607:f8b0:400b:802::200e, 172.217.165.14
Connecting to docs.google.com (docs.google.com)|2607:f8b0:400b:802::200e|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/21pho50n5gp9ai52bune1qru6u6832ug/1589912700000/14514704803973256873/*/1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu [following]
--2020-05-19 14:25:24--  https://doc-0k-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/21pho50n5gp9ai52bune1qru6u6832ug/1589912700000/14514704803973256873/*/1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu
Resolving doc-0k-0g-docs.googleusercontent.com (doc-0k-0g-docs.googleusercontent.com)... 2607:f8b0:400b:80f::2001, 172.217.164.193
Connecting to doc-0k-0g-docs.googleusercontent.com (doc-0k-0g-docs.googleusercontent.com)|2607:f8b0

In [39]:
def load_glove(glove_path, embedding_dim):
    with open(glove_path) as f:
        token_ls = [PAD_TOKEN, UNK_TOKEN]
        embedding_ls = [np.zeros(embedding_dim), np.random.rand(embedding_dim)]
        for line in f:
            token, raw_embedding = line.split(maxsplit=1)
            token_ls.append(token)
            embedding = np.array([float(x) for x in raw_embedding.split()])
            embedding_ls.append(embedding)
        embeddings = np.array(embedding_ls)
    return token_ls, embeddings

PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
EMBEDDING_DIM=300 # dimension of Glove embeddings
glove_path = "glove.6B.300d__50k.txt"
vocab, embeddings = load_glove(glove_path, EMBEDDING_DIM)

## Featurize

In [77]:
def featurize(data, labels, tokenizer, vocab, max_seq_length=128):
    vocab_to_idx = {word: i for i, word in enumerate(vocab)}
    text_data = []
    label_data = []
    for ex in tqdm.tqdm_notebook(data):
        tokenized = tokenizer.tokenize(ex.lower())
        ids = [vocab_to_idx.get(token, 1) for token in tokenized]
        text_data.append(ids)
    return text_data, labels
tokenizer = sacremoses.MosesTokenizer()
train_data_indices, train_labels = featurize(X_train, y_train, tokenizer, vocab)
val_data_indices, val_labels = featurize(X_dev, y_dev, tokenizer, vocab)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [79]:
print("\nTrain text first 5 examples:\n", train_data_indices[:5])
print("\nTrain labels first 5 examples:\n", train_labels[:5])


Train text first 5 examples:
 [[2, 567, 24, 16202, 16, 9, 3694, 5, 816, 1624, 1, 2, 38945, 22187, 16, 221, 21, 16, 2, 1624, 1, 55, 37, 1, 19, 2, 446, 1, 65, 34, 21516, 7223, 189, 102, 22, 1, 1073, 607, 6, 171, 9930, 4], [39, 335, 243, 8, 19733, 16, 1, 43, 42, 9, 10240, 12613, 7, 9, 2849, 5, 1, 2, 628, 5639, 287, 12, 199, 359, 2, 1705, 17, 3, 36, 129, 380, 3, 39, 16, 1, 2, 823, 88, 32, 9, 335, 1, 7, 16693, 3, 36, 2, 567, 16, 355, 3, 122, 3318, 1, 1545, 42865, 272, 4], [1300, 4155, 12, 406, 27, 16202, 78, 1, 15, 81, 3, 938, 1512, 7, 2, 567, 17, 1, 43, 35, 297, 22, 6, 2, 1387, 130, 4155, 676, 3, 21, 1404, 3980, 46, 6860, 4], [39, 16, 9, 3368, 24612, 335, 2553, 15, 9, 1924, 1, 85, 83, 1, 29410, 133, 19733, 206, 24823, 3, 39, 56, 32, 9, 355, 243, 6, 839, 12, 9, 1, 43, 1437, 2, 3691, 61, 2, 10240, 12613, 3, 102, 43, 42, 6, 843, 1, 43, 1, 193, 1753, 14, 43, 1, 361, 594, 12684, 13, 27, 2, 5874, 6, 2, 12907, 7, 9079, 3, 2, 10240, 7, 2, 6894, 13, 17, 1, 160, 135, 1571, 34, 145, 9272, 115, 5, 2,

## Create DataLoaders

In [80]:
class SpamDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list, max_sent_length=128):
        """
        @param data_list: list of data tokens 
        @param target_list: list of data targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        self.max_sent_length = max_sent_length
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key, max_sent_length=None):
        """
        Triggered when you call dataset[i]
        """
        if max_sent_length is None:
            max_sent_length = self.max_sent_length
        token_idx = self.data_list[key][:max_sent_length]
        label = self.target_list[key]
        return [token_idx, label]

    def spam_collate_func(self,batch):
        """
        Customized function for DataLoader that dynamically pads the batch so that all 
        data have the same length
        """ 
        data_list = [] # store padded sequences
        label_list = self.target_list #??? label list doesnt change 
        # find length of longest sequence 
        seq_lengths = [len(i) for i in self.data_list]
        max_batch_seq_len = max(seq_lengths) 

        if max_batch_seq_len >= self.max_sent_length:
            max_batch_seq_len = self.max_sent_length 

        for seq in self.data_list: 
            #pad short sentences
            if len(seq) < max_batch_seq_len:
                seq += [int(0)] * (max_batch_seq_len - len(seq))
            #trim long sentences
            elif len(seq) > max_batch_seq_len:
                seq = seq[:max_batch_seq_len]
            else: pass

            assert(len(seq) == max_batch_seq_len)
            data_list.append(seq)
        
        assert(len(data_list) == len(label_list))
        
        #data_list = torch.Tensor(data_list)
        #label_list = torch.Tensor(label_list)
        
        data_list = torch.from_numpy(np.array(data_list))
        label_list = torch.LongTensor(label_list)
        label_list = label_list.long()
        
        return [data_list, label_list]

BATCH_SIZE = 64
max_sent_length=128
train_dataset = SpamDataset(train_data_indices, train_labels, max_sent_length)

val_dataset = SpamDataset(val_data_indices, val_labels, train_dataset.max_sent_length)



In [82]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=train_dataset.spam_collate_func,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=train_dataset.spam_collate_func,
                                           shuffle=False)


In [86]:
data_batch, labels = next(iter(train_loader))

print("data batch dimension: ", data_batch.size())
print("data_batch: ", data_batch)
print("labels: ", labels)

data batch dimension:  torch.Size([250874, 128])
data_batch:  tensor([[   2,  567,   24,  ...,    0,    0,    0],
        [  39,  335,  243,  ...,    0,    0,    0],
        [1300, 4155,   12,  ...,    0,    0,    0],
        ...,
        [  88,    1,  205,  ...,    0,    0,    0],
        [  42,    9,  355,  ...,    0,    0,    0],
        [9390, 6743, 9538,  ...,  319,  183,   47]])
labels:  tensor([1, 1, 1,  ..., 0, 0, 0])


In [89]:
data_batch

tensor([[   2,  567,   24,  ...,    0,    0,    0],
        [  39,  335,  243,  ...,    0,    0,    0],
        [1300, 4155,   12,  ...,    0,    0,    0],
        ...,
        [  88,    1,  205,  ...,    0,    0,    0],
        [  42,    9,  355,  ...,    0,    0,    0],
        [9390, 6743, 9538,  ...,  319,  183,   47]])

## Build a simple BiLSTM Classifier 

In [90]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMClassifier(nn.Module):
    """
    LSTMClassifier classification model
    """
    def __init__(self, embeddings, hidden_size, num_layers, num_classes, bidirectional, dropout_prob=0.3):
        super().__init__()
        self.embedding_layer = self.load_pretrained_embeddings(embeddings)
        self.dropout = None
        self.lstm = nn.LSTM(input_size=embeddings.shape[1],hidden_size=hidden_size,bidirectional=bidirectional,dropout=dropout_prob,batch_first=True,num_layers=num_layers)
        self.non_linearity = nn.ReLU() # For example, ReLU
        self.clf = nn.Linear(self.embedding_layer.embedding_dim,num_classes) # classifier layer
        self.sig = nn.Sigmoid()
        """
           define the components of your BiLSTM Classifier model
           You may refer to Lab2 for reference
           2. TODO: Your code here
        """
        
    
    def load_pretrained_embeddings(self, embeddings):
        """
           The code for loading embeddings from Lab 2
           Unlike lab, we are not setting `embedding_layer.weight.requires_grad = False`
           because we want to finetune the embeddings on our data
        """
        embedding_layer = nn.Embedding(embeddings.shape[0], embeddings.shape[1], padding_idx=0)
        embedding_layer.weight.data = torch.Tensor(embeddings).float()
        return embedding_layer


    def forward(self, inputs):
        logits = None
        """
           Write forward pass for LSTM
           Example, forward:= embedding -> bilstm -> pooling (sum?mean?max?) 
                              nonlinearity -> classifier
           Refer to: https://arxiv.org/abs/1705.02364 
           Return logits
           You may refer to Lab2 for embedding lookup and how to return logits
           3. TODO: Your code here
        """
        ## pass inputs to embedding layer
        embedded = self.embedding_layer(inputs)


        num_embedded = (inputs != self.embedding_layer.padding_idx) \
            .float().sum(dim=1).clamp(1)
          
        averaged_embeddings = embedded.sum(dim=1)/num_embedded.view(-1,1)
        
       ## max pooling => torch.max()
        
        #averaged_embeddings = self.non_linearity(averaged_embeddings)
        out = self.clf(averaged_embeddings)
        
        
        logits = self.sig(out)
        
        return logits

        

In [91]:
def evaluate(model, dataloader, device):
    accuracy = None
    model.eval()
    """
        4. TODO: Your code here
        Calculate the accuracy of the model on the data in dataloader
        You may refer to `run_inference` function from Lab2 
    """
    correct = 0
    total = 0
    test_loss = 0

    #https://github.com/pytorch/examples/blob/master/mnist/main.py
    with torch.no_grad():
        for data_batch, labels_batch in test_loader:
            data_batch, labels_batch = data_batch.to(device), labels_batch.to(device)
            output = model(data_batch)
            test_loss += F.nll_loss(output, labels_batch, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(labels_batch.view_as(pred)).sum().item()
            
    accuracy = correct/len(test_loader.dataset)
    #print(correct, len(test_loader.dataset))
    return accuracy

## Initialize BiLSTM model, criterion and optimizer

In [ ]:
# BiLSTM hyperparameters
hidden_size = 32
num_layers = 3
num_classes = 2
bidirectional=True
torch.manual_seed(1234)

# if cuda exists, use cuda, else run on cpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device=torch.device('cpu')

model = LSTMClassifier(embeddings, hidden_size, num_layers, num_classes, bidirectional)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## Train model with early stopping

In [ ]:
train_loss_history = []
val_accuracy_history = []
best_val_accuracy = 0
n_no_improve = 0
early_stop_patience=2
NUM_EPOCHS=10
  
for epoch in tqdm.tqdm_notebook(range(NUM_EPOCHS)):
    model.train() # this enables regularization, which we don't currently have
    for i, (data_batch, batch_labels) in enumerate(train_loader):
        """
           Code for training lstm
           Keep track of training of for each batch using train_loss_history
        """
        preds = model(data_batch.to(device))
        loss = criterion(preds, batch_labels.to(device))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss_history.append(loss.item())
    # The end of a training epoch 

    """
        Code for tracking best validation accuracy, saving the best model, and early stopping
        # Compute validation accuracy after each training epoch using `evaluate` function
        # Keep track of validation accuracy in `val_accuracy_history`
        # save model with best validation accuracy, hint: torch.save(model, 'best_model.pt')
        # Early stopping: 
        # stop training if the validation accuracy does not improve for more than `early_stop_patience` runs
        5. TODO: Your code here
    """
    val_accuracy = evaluate(model,test_loader,device)
    val_accuracy_history.append(val_accuracy)
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
      ## save as new best model
        torch.save(model, 'best_model.pt')
    else:
        n_no_improve += 1
        if n_no_improve > early_stop_patience:
            print("Stopped at",n_no_improve,"iterations.")
            break  
print("Best validation accuracy is: ", best_val_accuracy)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
